In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [4]:
df = fetch_hourly_rides(12)

2025-05-10 19:34:04,695 INFO: Initializing external client
2025-05-10 19:34:04,695 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 19:34:05,444 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 


In [5]:
df

,pickup_hour,pickup_location_id,rides
0,2025-05-10 15:00:00+00:00,HB402,0
1,2025-05-10 18:00:00+00:00,HB506,7
2,2025-05-10 10:00:00+00:00,JC109,0
3,2025-05-10 16:00:00+00:00,JC024,0
4,2025-05-10 08:00:00+00:00,HB105,0
...,...,...,...
991,2025-05-10 10:00:00+00:00,JC097,0
992,2025-05-10 13:00:00+00:00,JC032,0
993,2025-05-10 15:00:00+00:00,JC051,0
994,2025-05-10 15:00:00+00:00,JC097,0


In [6]:
df_pred = fetch_predictions(12)

2025-05-10 19:34:26,143 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 19:34:26,145 INFO: Initializing external client
2025-05-10 19:34:26,145 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 19:34:26,775 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.43s) 


In [7]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,JC104,2.0,2025-05-11 00:00:00+00:00
1,HB608,1.0,2025-05-11 00:00:00+00:00
2,JC020,1.0,2025-05-11 00:00:00+00:00
3,HB302,2.0,2025-05-11 00:00:00+00:00
4,JC077,2.0,2025-05-11 00:00:00+00:00
...,...,...,...
78,JC081,1.0,2025-05-11 00:00:00+00:00
79,HB409,0.0,2025-05-11 00:00:00+00:00
80,JC074,1.0,2025-05-11 00:00:00+00:00
81,HB404,1.0,2025-05-11 00:00:00+00:00


In [8]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [9]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand


In [10]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [11]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [12]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [13]:
mae_by_hour["MAE"].mean()

nan

In [16]:
def fetch_hourly_ridess(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor("h")
    current_hour_str = current_hour.strftime('%Y-%m-%d %H:%M:%S') 
   
    print(current_hour)
    print(type(current_hour))
    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_NAME, version=1)

    query = fg.select_all()
    print(query.read())
    print(fg.pickup_hour)
    

    return query.read()

df=fetch_hourly_ridess(12)

2025-05-10 11:00:00+00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2025-05-10 19:36:44,821 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 19:36:44,823 INFO: Initializing external client
2025-05-10 19:36:44,823 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 19:36:45,556 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.78s) 
                     pickup_hour pickup_location_id  rides
0      2025-05-03 20:00:00+00:00              JC108      0
1      2025-04-14 07:00:00+00:00              JC009      0
2      2025-04-19 05:00:00+00:00              HB301      0
3      2025-05-02 08:00:00+00:00              JC075      0
4      2025-05-02 12:00:00+00:00              JC053      2
...                          ...                ...    ...
113705 2024-05-11 21:00:00+00:00              JC108   

In [17]:
df=fetch_hourly_ridess(12)
df

2025-05-10 11:00:00+00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2025-05-10 19:36:58,896 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 19:36:58,898 INFO: Initializing external client
2025-05-10 19:36:58,899 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 19:36:59,594 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.20s) 
                     pickup_hour pickup_location_id  rides
0      2025-05-03 20:00:00+00:00              JC108      0
1      2025-04-14 07:00:00+00:00              JC009      0
2      2025-04-19 05:00:00+00:00              HB301      0
3      2025-05-02 08:00:00+00:00              JC075      0
4      2025-05-02 12:00:00+00:00              JC053      2
...                          ...                ...    ...
113705 2024-05-11 21:00:00+00:00              JC108   

,pickup_hour,pickup_location_id,rides
0,2025-05-03 20:00:00+00:00,JC108,0
1,2025-04-14 07:00:00+00:00,JC009,0
2,2025-04-19 05:00:00+00:00,HB301,0
3,2025-05-02 08:00:00+00:00,JC075,0
4,2025-05-02 12:00:00+00:00,JC053,2
...,...,...,...
113705,2024-05-11 21:00:00+00:00,JC108,0
113706,2024-05-11 21:00:00+00:00,JC076,1
113707,2024-05-11 21:00:00+00:00,JC105,1
113708,2024-05-11 21:00:00+00:00,HB303,1


In [19]:
current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=12)).floor("h")
current_hour_str = current_hour.strftime('%Y-%m-%d %H:%M:%S') 
df.sort_values(by=["pickup_hour"], ascending=False)

,pickup_hour,pickup_location_id,rides
26517,2025-05-10 18:00:00+00:00,JC094,0
162,2025-05-10 18:00:00+00:00,HB506,7
37928,2025-05-10 18:00:00+00:00,JC018,1
11492,2025-05-10 18:00:00+00:00,HB503,1
42695,2025-05-10 18:00:00+00:00,HB401,1
...,...,...,...
88944,2024-04-12 21:00:00+00:00,JC076,2
77108,2024-04-12 21:00:00+00:00,JC116,0
99394,2024-04-12 21:00:00+00:00,JC059,0
74564,2024-04-12 21:00:00+00:00,JC110,2
